In [90]:
import pandas  # used for importing the .csv files
import datetime # used for dates
import numpy
filea = ('games2005.csv','games2006.csv','games2007.csv','games2008.csv','games2009.csv','games2010.csv','games2015.csv')
fileb = ('nba04.csv','nba05.csv','nba06.csv','nba07.csv','nba08.csv','nba09.csv','nba14.csv')
for i in range(1,7):
    games = pandas.read_csv(filea[i])
    lines = pandas.read_csv(fileb[i])
    games['line']=0
    games['game_date']=0
    lines['game_date']=0
    games['hscore']=games['PTS.1']
    games['rscore']=games['PTS']
    for idx,rows in games.iterrows():
        games.loc[idx,'game_date'] = datetime.datetime.strptime(games.loc[idx]["Date"],"%a, %b %d, %Y")
    for idx,rows in lines.iterrows():
        lines.loc[idx,'game_date'] = datetime.datetime.strptime(lines.loc[idx]["date"],"%m/%d/%Y")
    s1 = pandas.merge(games, lines, how = 'left', on=['game_date','hscore','rscore'])
    s1.to_csv(filea[i])

In [86]:
import pandas
import numpy as np
import datetime
import matplotlib.pyplot as plt
elo_spd = []
pt_spd = []
def score_elo(hp1,hp2,hp3,hp4,csv): # define a new function that takes in hyperparameter values and returns the percent of games correctly predicted
    data = pandas.read_csv(csv)
    ID = {}
    outcomes = []

    for idx, row in data.iterrows():
        if data.loc[idx,'PTS']>data.loc[idx,'PTS.1']:
            winner = data.loc[idx,'Visitor/Neutral']; loser = data.loc[idx,'Home/Neutral'];
            pd = data.loc[idx,'PTS'] - data.loc[idx,'PTS.1']; hm_i = 0
        else:
            winner = data.loc[idx,'Home/Neutral']; loser = data.loc[idx,'Visitor/Neutral'];
            pd = data.loc[idx,'PTS.1'] - data.loc[idx,'PTS'];hm_i = 1
        game_date = datetime.datetime.strptime(data.loc[idx]["Date"],"%a, %b %d, %Y")
        if winner in ID.keys():
            w_elo = ID[winner][0]
            if (ID[winner][1] - game_date).days == 1: w_date = 1 
            else: w_date = 0
            w_games = ID[winner][2]
        else: 
            w_elo = 1500
            w_date = 0
            w_games = 0
        if loser in ID.keys():
            l_elo = ID[loser][0]
            if (ID[loser][1] - game_date).days == 1: l_date = 1
            else: l_date = 0
            l_games = ID[loser][2]
        else: 
            l_elo = 1500
            l_date = 0
            l_games = 0
        if hm_i == 1: w_elo_g = w_elo+hp3 + hp4*w_date; l_elo_g = l_elo + hp4*l_date
        else: l_elo_g = l_elo +hp3 + hp4*l_date; w_elo_g = w_elo + hp4*l_date;
        dif = w_elo_g - l_elo_g
        if dif>=0:
            pt_spd.append(pd)
            elo_spd.append(dif)
        else:
            pt_spd.append(-pd)
            elo_spd.append(-dif)
        if w_elo_g>l_elo_g: outcomes.append(1)
        if l_elo_g>w_elo_g: outcomes.append(0)
        w_new_elo = (w_elo+np.log(pd+1)*(2.2/((w_elo-l_elo)*.001+2.2))*hp1*(1-(1/(1+10**((l_elo_g-w_elo_g)/400)))))
        l_new_elo = (l_elo+np.log(pd+1)*(2.2/((w_elo-l_elo)*.001+2.2))*hp1*(0-(1/(1+10**((w_elo_g-l_elo_g)/400)))))
        ID[winner]=(w_new_elo,game_date,w_games + 1);
        ID[loser]=(l_new_elo,game_date,l_games + 1)
    print (sum(outcomes)/float(len(outcomes)))
score_elo(10,700,10,-.5,'games2015.csv')# run the function on the 2015 data with the given hyperparameters
print (np.polyfit(elo_spd,pt_spd,1))

0.6772357723577236
[ 0.03433173  1.58332637]


In [63]:
print (np.polyfit(elo_spd,pt_spd,1)[1])

2.36430226426


In [87]:
elo_pred= [elo_spds*np.polyfit(elo_spd,pt_spd,1)[0] + np.polyfit(elo_spd,pt_spd,1)[1] for elo_spds in elo_spd]
se= [np.float64((elo_preds - pt_spds)).item()**2.0 for elo_preds,pt_spds in zip(elo_pred,pt_spd)]
print (sum(se)/len(se))
print (sum(se))


154.49039493068128
190023.18576473798


In [82]:
A = np.power.outer(elo_spd,np.arange(1,len(elo_spd))) 
outputs = np.linalg.lstsq(A,pt_spd) 

LinAlgError: SVD did not converge in Linear Least Squares